In [1]:
import tweepy as tw
import datetime as dt
import pandas as pd
from keras.models import load_model
import numpy as np
from pythainlp import word_tokenize
from keras.preprocessing.sequence import pad_sequences

import re

In [2]:
# pip install tweepy

In [3]:
def complete_func_v2(input_from_user):
    
    
    # access token key and access secret key => Suphachai Sriwised  ใช้ของใครก้ได้ เอามายืนยันตัวตนให้ api ทำงานได้เฉยๆ
    access_token = '247327218-2Y9OmrkjTGBtyKv91zJl7ljwPeEUhkDJIC2lxMnB'
    access_secret_key = 'Ygy2CPjPaOY3P88UFbmazzNzRU60CviTsJe99VPvlWnBF'
    
    

    # อันนี้ต้องใช้ของคนที่ได้รับสิทธิ twitter api developper เท่านั้น ถึงจะทำงานได้
    consumer_key = 'V6WhIk4MThLDHOZffp5ePjhxb'
    consumer_secret_key = '0mBgQEhDmlCJYFiEuzv49bwauCKzvBHh7yg2ph5QksMNxHTaOQ'
    
    
    
    # ยืนยันตัวตนเพื่อเข้าใช้งาน twitter API    
    auth = tw.OAuthHandler(consumer_key, consumer_secret_key)
    auth.set_access_token(access_token, access_secret_key)
    api = tw.API(auth, wait_on_rate_limit=True)
    
    
    
    # ใช้ method ของ library tweepy เพื่อค้นหาตามคีย์เวิร์ด ซึ่งสามารถกรอกคำสั่งตามที่มีใน doc ของ twitter API ได้เพื่อตัด หรือ ค้น ในสิ่งที่ต้องการ 
    # คำสั่งจะใช้ตามนี้ https://developer.twitter.com/en/docs/twitter-api/v1/rules-and-filtering/overview/standard-operators    
    tweets = tw.Cursor(api.search,
              q=input_from_user+' -filter:retweets',
              lang="th",
              since="").items(500)
    
    
    
    # ทำให้เป็น csv (แบบยังไม่มีผลการทำนาย)     
    step1 = to_csv_v2(tweets)
    print(step1)
    
    
    
# ข้ามการ normalize ไปก่อนเพราะเด่วเราจะไปทำที่ auto label ถ้าทำจากอันนั้นจะได้ ออกมาเป็น word,toxic พร้อมใช้งาน 
# แต่อันนี้ต้องให้คนตรวจอีกรอบก่อน เพราะมันมีสามช่อง  
    #ทำการประกาศตัวแปรเป็นโกลบอลเพื่อให้ normaliz_v1 (จะมีสองเวอร์ v1 จะเป็นแบบลูป v2 จะเป็นแบบที่ละอัน) เรียกใช้งานได้ (ความจริงจะประกาศใน func นั้นก้ได้แต่ ขก.ล้วนๆ)
#     global clawing_model , dict_fileWord , nonePredict_documents
#     clawing_model = load_model('cnn&rnn_model.h5')
#     dict_fileWord=pd.read_csv('dict_file.csv')
#     nonePredict_documents = pd.read_csv ('dataset_tweet.csv')
#     print('this is none predict document csv file : ',test_documents)
    

#     # normalize จะแปลงข้อมูลให้อยู่ในรูปแบบพร้อมใช้งานสำหรับเทรนโมเดล  
#     step2 = normalize_v1(nonePredict_documents)
#     print(step2)
    
    
    
#     completePredict_file = pd.read_csv ('./new_csv.csv')
#     # complete_file
# #     print(completePredict_file)
    
    
    
    return 'complete step to csv an'




#         for tweet in tweets:
#             print(tweet.text)
#     data_tw  = [[tweet.text] for tweet in tweets] 

#         users_locs = [[tweet.user.screen_name, tweet.user.location,tweet.text] for tweet in tweets]
#         users_locs
        
#     tweet_text = pd.DataFrame(data=data_tw, 
#                         columns=["tweet"])
#     tweet_text.to_csv('dataset_beta_1.0.csv',index=False,encoding='utf-8-sig')
#     # ทำการค้นหาคีย์เวิร์ด    

In [4]:
# อันใหม่ tweepy 
# แบบสามแถว
def to_csv_v2(tw):    
        
        # ทำการอ่านไฟล์เก่า เพื่อที่จะเอาข้อมูลจากไฟล์เก่ามาเก็บในลิสต์
        old_csv = pd.read_csv('dataset_from_scrapper/dataset_tweet.csv');
        
        #ถ้ามีในไฟล์เก่าก่อนหน้า ก้จะวนลูปเพื่อเอาค่าเก่ามาด้วย มันจะได้ไม่ทับอันเก่า
        if(len(old_csv ) !=0 ):
            #สร้างลิสต์เปล่ามารองรับข้อมูลจากไฟล์เก่าที่เราดึงมา
            list_tw = []
            
            for word in old_csv['word']:
                # RegEx                  
                result_1 = re.sub(r"http\S+", "",word)
                # ตัว r จะทำให้พวก /n และอื่นๆไม่ทำงาน                
                result_2 = re.sub(r"@\w+","",result_1)
                result_3 = re.sub(r"#\w+","",result_2)
                result_4 = re.sub(r"\a","",result_3)
                list_tw.append(result_4)
        #print อันนี้เอาไว้ debug เฉยๆ
        #print('get old list_tw:',list_tw) 
        
        #ถ้าไม่มีจะสร้างลิสต์เปล่าๆเพื่อเตรียมนำ tweet ที่พึ่งได้มาใส่     
        else:
            #ถ้าเข้าเคสนี้แสดงว่าไฟล์ก่อนหน้ามันว่างเปล่า
            list_tw = []
            
        #จาก data ที่เราได้จากการใช้ tweepy จะออกมาเป็นในรูปแบบของ object เราจะวนลูปเพื่อดึงข้อมูลที่เป็น text ออกมา หรือก็คือ tweetที่หาเจอ 
        for t in tw:
            result_1 = re.sub(r"http\S+", "",t.text)
            
            result_2 = re.sub(r"@\w+","",result_1)
            result_3 = re.sub(r"#\w+","",result_2)
            result_4 = re.sub(r"\a","",result_3)
            #วนลูปเก็บเข้าไปที่ลิสต์ที่เราสร้างมารองรับก่อนหน้า
            list_tw.append(result_4);
        
        #print อันนี้เอาไว้ debug เฉยๆ
        #print('list_tw:',list_tw)
        
        #ทำการสร้าง dataframe ขึ้นมา ตามรูปแบบที่เราต้องการคือ word,toxic1,toxic2,toxic3 
        df = pd.DataFrame({'word':list_tw,
                            'toxic1':None,
                           'toxic2':None,
                           'toxic3':None})
        
        #ต่อมานำ dataframe ที่สร้างมาจัดเก็บเป็น csv เพื่อนำไปใช้งานในส่วนงานอื่นๆ
        df.to_csv('dataset_from_scrapper/dataset_tweet.csv',index=False,encoding='utf-8-sig')
        return len(pd.read_csv('dataset_from_scrapper/dataset_tweet.csv'))

# เตรียมข้อมูลให้มีรูปแบบที่ใช้ได้ต่อ

In [ ]:
from flask import Flask,render_template,request,url_for

app = Flask(__name__)


@app.route('/')
def index():
    return render_template('/main.html')

@app.route('/save',methods=['GET','POST'])
def save():

    #รับ req เป็นแบบ post เข้ามา
    if request.method == 'POST':

        #นำคำที่ user ใส่เข้ามาเก็บเอาไว่ที่ keyword_search
        keyword_search = request.form['keyword']

        # print keyword ออกมาดูเพื่อ debug    
        print('this is keyword =>>> ', keyword_search)
        
        #ทำการเรียก function นี้จากนั้น function นี้จะทำการ call function อื่นๆเพื่อทำงานต่อไป
        step3 = complete_func_v2(keyword_search)
        print(step3)
        
        #ทำการแจ้งเตือนผ่าน alert.html เมื่อทำงานเสร็จสิ้นแล้ว
        return render_template('alert.html')
    else:
        #แจ้ง error เมื่อส่ง req แบบอื่นๆเข้ามา
        return 'error 400 please use post method'


if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [01/Nov/2020 19:09:10] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [01/Nov/2020 19:09:10] "GET /favicon.ico HTTP/1.1" 404 -


In [10]:
Data = pd.read_csv('dataset_from_scrapper/dataset_tweet.csv')
Data

,word,toxic1,toxic2,toxic3
0,หวั่นระบาดซ้ำ! ฮ่องกง ขยายเวลาเว้นระยะห่างป้อง...,NaN,NaN,NaN
1,Suzu Hirose นักแสดงชาวญี่ปุ่นได้รับการยืนยันว่...,NaN,NaN,NaN
2,ัดพระธรรมกาย&amp;มูลนิธิธรรมกาย\nมอบความช่วยเ...,NaN,NaN,NaN
3,ตะลึง! องค์การอนามัยโลก (WHO) ชี้ผู้ติดเชื้อ ิ...,NaN,NaN,NaN
4,อย่าลืมสวมหน้ากาก รักษาระยะห่าง หมั่นล้างมือ น...,NaN,NaN,NaN
...,...,...,...,...
6851,ช่วยกันค่ะ ๊อป18ตุลา ๊อป14ตุลา,NaN,NaN,NaN
6852,เกลียดเผด็จการ ีดเส้นใต้ไล่เผด็จการ ็อบ18ตุลา,NaN,NaN,NaN
6853,ทำไงดีวะกูอยู่ที่บ้านสลิ่มนั่งฟังจนป่วยจิตหมดล...,NaN,NaN,NaN
6854,"""ไม่รักพ่อ ออกจากบ้านพ่อไป"" อ๋อ ออกไม่ได้ค่ะ ช...",NaN,NaN,NaN


In [4]:
# # เป็นส่วนเตรียมข้อมูลให้พร้อมสำหรับการ predict
# def getinput(train):
    
#     #เป็นลิสต์สำหรับเก็บคำที่ตัดของแต่ละประโยคที่เข้ามา     
#     list_word = []
    
#     #ทำการตัดประโยคเป็นคำด้วย word_tokenize ของ pythainlp ที่ import เข้ามา     
#     proc = word_tokenize(train, engine='newmm')
    
#     # เก็บคำที่ตัดเอาไว้ที่ลิสต์    
#     list_word.append(proc)
    
#     #สร้างลิสต์สำหรับเก็บลำดับของแต่ละคำภายใน list_word 
#     sequence = list()
    
#     for n in list_word: #ลูปเปลี่ยน คำ string ที่ตัดมาแปลงเป็นตัวเลข
#         #เป็นตัวเก็บลำดับชั่วคราวจะรีเซ็ตทุกๆรอบ         
#         sequenceTemp = list() 
        
#         for w in n:
#             count=0
#             for x in dict_fileWord['Unnamed: 0']: #ลูปเทียบdictคำ
#                 count=count+1
#                 if x==w:
#                     sequenceTemp.append(count)
#                     break
#         sequence.append(sequenceTemp)  
        
#     test = np.array(sequence)

#     testX = pad_sequences(test, maxlen=105, padding="post") #input_length=105
#     return testX

In [5]:
# # ตอนนี้ยังไม่ใช้นะ ไปใช้ที่ autolabel ที่เดียว
# def normalize_v1(data):
#     # กระบวนการหาผลลัพธ์ เตรียมใส label อัตโนมัติ
# #     print(data['word'])
#     label_toxic1=[]
#     label_toxic2=[]
#     label_toxic3=[]
    
#     #รับ dataframe ทั้งอันเข้ามา เข้าถึงคอลัมป์ word
#     for x in data['word']:
           
#         list_label_toxic = [] #สำหรับเก็บค่า index ที่เป็น multilabel
        
#         # เตรียมอินพุตสำหรับการ predict 
#         test=getinput(x)
#         #print(test)
        
#         #เรียกโมเดลออกมาใช้ predict
#         ynew = clawing_model.predict(test)
        
#         # สร้างลิสต์ออกมาเพื่อเก็บผลการทำนาย    
#         yn = []
        
#         #วนลูปใน ynew เพื่อใช้งานค่าพยากรณ์ 
#         for b in ynew[0]:
            
#             #แปลงเลขให้อยู่ในทศนิยม 4 ตำแหน่ง
#             num_per = '%.4f' % b
            
#             #เก็บเลขที่แปลงเข้าไป
#             yn.append(num_per)
        
#         #  print(ynew[0])
#         #หาค่าที่มากที่สุดเพื่อเป็นตัวเทียบ multilabel         
#         max_ynew = max(ynew[0][:])
# #         print('max :-- ','%.4f' % max_ynew)

#         #คิดค่าจาก 10 เปอร์ของตัดมากสุด เพื่อเทียบความใกล้เคียงของคะแนน
#         tenPer_max_ynew = max_ynew * 0.9
# #         print('tenper --','%.4f' % tenPer_max_ynew)
# #         print('result_predict :====',yn)
# #                                                                                      passed


#         list_label=[] #เก็บค่าที่เป็น Multilabel ไว้ที่นี้
#         for i in yn:

#             #print('multilabel :++++++++++++',i)
#             if(i >= '%.4f' %tenPer_max_ynew ):
#                 list_label.append(i)
# #         print('ผลลัพธ์จากการทำนาย Multilabel ::::::::::',list_label)                                 #passed 

#     #     เก็บตำแหน่งที่เป็น multilabel โดยการเทียบกับ yn
#         for ii in list_label:
            
#             count = 0 
            
#             for iii in yn:
                
#                 count+=1
                
#                 if(ii == iii):
#                     list_label_toxic.append(count)
#                     break
# #         print('ตำแหน่งของ Multilabel จากที่ทำนาย ::::::::::',list_label_toxic)                      #passed 

#         count_j=0
#         count_len=0
#         for j in list_label_toxic:# มีขนาดตามตำแหน่งที่รับมา
#             count_j+=1

#             if(count_j==1):
# #                 print('Index_multilabel::::::::::',list_label_toxic[0],'lenght :::: 1')
#                 label_toxic1.append(list_label_toxic[0])
#                 count_len=1

#             if(count_j==2):
# #                 print('Index_multilabel::::::::::',list_label_toxic[0:2],'lenght :::: 2')
#                 label_toxic2.append(list_label_toxic[1])
#                 count_len=2

#             if(count_j==3):
# #                 print('Index_multilabel::::::::::',list_label_toxic[0:3],'lenght :::: 3')
#                 label_toxic3.append(list_label_toxic[2])
#                 count_len=3
#                                                                                     #passed 
#     # แทนที่ตำแหน่งที่ไม่มีด้วย 0
#         if( count_len==1):
#             label_toxic2.append(0)
#             label_toxic3.append(0)
#         if( count_len==2):
#             label_toxic3.append(0)
#     #    ตรวจตำแหน่งอีกรอบด้วยตาตามแนวตั้ง     
# #     print(label_toxic1,label_toxic2,label_toxic3) 

#     # เตรียมทำ csv แบบที่มีการทำนายผลแล้วเรียบร้อย
#     df = pd.DataFrame({'word':nonePredict_documents['word'],
#                                     'toxic1': label_toxic1,
#                                     'toxic2': label_toxic2,
#                                     'toxic3': label_toxic3})

#     df.to_csv('new_csv.csv',index=False,encoding='utf-8-sig')
# #     label_documents = pd.read_csv ('new_csv.csv')

# #     print('label_document : ',label_documents)
#     return 'normalize complete'
        

In [6]:
#ของเก่าเก็บไว้ดูก่อน
# def complete_func(input_from_user):
    
#     # ทำการค้นหาคีย์เวิร์ด    
#     tw = query_tweets(input_from_user,limit=None)
    
#     # export to dataframe and make this to csv แบบสามแถว
#     to_csv_v2(tw)
    
#     global clawing_model
#     clawing_model = load_model('cnn&rnn_model.h5')

#     global dict_fileWord
#     dict_fileWord=pd.read_csv('dict_file.csv')
    
#     global test_documents
#     test_documents = pd.read_csv ('dataset_tweet.csv')
#     # test_documents
    
#     normalize_v1(test_documents)
    
#     complete_file = pd.read_csv ('./new_csv.csv')
#     # complete_file
#     print(complete_file)

In [2]:
# อันเก่าเก็บไว้ดูก่อน
# # แบบสามแถว
# def to_csv_v2(tw):    
#     # read previous csv file before append the new word
#         old_csv = pd.read_csv('dataset_tweet.csv');
#         if(len(old_csv ) !=0 ):
#             list_tw = []
#             for word in old_csv['word']:
#                 list_tw.append(word)
#             print('get old list_tw:',list_tw)
#         else:
#             # make new list to save text from  tweet object
#            # only first time to create csv file    
#             list_tw = []
#         # loop on list of tweet object
#         for t in tw:
#             list_tw.append(t.text);
#         print('list_tw:',list_tw)
#         df = pd.DataFrame({'word':list_tw,
#                             'toxic1':None,
#                            'toxic2':None,
#                            'toxic3':None})

#         df.to_csv('dataset_tweet.csv',index=False,encoding='utf-8-sig')